# review_card

> Review card component showing assembled segment with timing and source info

In [ ]:
#| default_exp components.review_card

In [ ]:
#| export
from typing import Any, Callable, Optional

from fasthtml.common import Div, Span, P

# DaisyUI components
from cjm_fasthtml_daisyui.components.data_display.badge import badge, badge_styles
from cjm_fasthtml_daisyui.components.data_display.card import card, card_body
from cjm_fasthtml_daisyui.components.feedback.loading import loading, loading_styles, loading_sizes
from cjm_fasthtml_daisyui.utilities.semantic_colors import bg_dui, text_dui

# Tailwind utilities
from cjm_fasthtml_tailwind.utilities.spacing import p, m
from cjm_fasthtml_tailwind.utilities.sizing import w, min_h
from cjm_fasthtml_tailwind.utilities.typography import (
    font_size, font_weight, font_family, leading
)
from cjm_fasthtml_tailwind.utilities.layout import position, right, top, visibility
from cjm_fasthtml_tailwind.utilities.transforms import translate
from cjm_fasthtml_tailwind.utilities.effects import opacity
from cjm_fasthtml_tailwind.utilities.transitions_and_animation import transition, duration
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import (
    flex_display, flex_direction, items, justify, gap, grow, shrink
)
from cjm_fasthtml_tailwind.core.base import combine_classes

# Card stack library
from cjm_fasthtml_card_stack.core.constants import CardRole
from cjm_fasthtml_card_stack.core.models import CardRenderContext

# Segmentation and alignment models
from cjm_transcript_segmentation.models import TextSegment
from cjm_transcript_vad_align.models import VADChunk

# Local imports
from cjm_transcript_review.html_ids import ReviewHtmlIds
from cjm_transcript_review.utils import format_time, format_duration, format_source_info

## AssembledSegment

Pairs a TextSegment with its corresponding VADChunk for display.

In [ ]:
#| export
from dataclasses import dataclass

@dataclass
class AssembledSegment:
    """A segment paired with its corresponding VAD chunk for review."""
    
    segment: TextSegment  # Text segment with content and source info
    vad_chunk: VADChunk  # VAD chunk with timing
    
    @property
    def index(self) -> int:  # Segment index for card stack
        """Get the index from the segment."""
        return self.segment.index
    
    @property
    def text(self) -> str:  # Segment text content
        """Get the text content from the segment."""
        return self.segment.text
    
    @property
    def start_time(self) -> float:  # Start time from VAD chunk
        """Get the start time from the VAD chunk."""
        return self.vad_chunk.start_time
    
    @property
    def end_time(self) -> float:  # End time from VAD chunk
        """Get the end time from the VAD chunk."""
        return self.vad_chunk.end_time

## Card Renderer

Each review card shows the assembled segment data:
- Index badge
- Text content
- Time range and duration
- Source reference

In [ ]:
#| export
def render_review_card(
    assembled:AssembledSegment,  # Assembled segment with text and timing
    card_role:CardRole,  # Role of this card in viewport ("focused" or "context")
) -> Any:  # Review card component
    """Render a single review card with text, timing, and source info."""
    is_focused = card_role == "focused"
    seg = assembled.segment
    chunk = assembled.vad_chunk
    
    # Text styling based on card role
    text_cls = text_dui.base_content if is_focused else text_dui.base_content.opacity(70)
    meta_opacity = opacity(60) if is_focused else opacity(40)
    
    # Time range display
    time_range = Span(
        f"{format_time(chunk.start_time)} \u2192 {format_time(chunk.end_time)}",
        cls=combine_classes(font_size.xs, font_family.mono, meta_opacity)
    )
    
    # Duration badge
    duration_badge = Span(
        format_duration(chunk.start_time, chunk.end_time),
        cls=combine_classes(badge, badge_styles.ghost, font_size.xs, font_family.mono)
    )
    
    # Source info
    source_info = Span(
        format_source_info(
            seg.source_provider_id,
            seg.source_id,
            seg.start_char,
            seg.end_char
        ),
        cls=combine_classes(font_size.xs, font_family.mono, meta_opacity)
    )
    
    # Playing indicator — hidden by default, toggled visible by JS during audio playback
    playing_indicator = Div(
        Span(cls=combine_classes(loading, loading_styles.bars, loading_sizes.xs, text_dui.secondary)),
        cls=combine_classes(
            "review-playing-indicator",
            position.absolute, right(2), top("1/2"), translate.y("1/2").negative,
            visibility.invisible,
        )
    )
    
    return Div(
        Div(
            # Left column: Index and metadata
            Div(
                # Index badge
                Span(
                    f"#{seg.index + 1}",
                    cls=combine_classes(
                        font_size.xs, font_family.mono, font_weight.bold,
                        opacity(50)
                    )
                ),
                cls=combine_classes(w(12), shrink(0))
            ),
            
            # Center: Main content
            Div(
                # Text content
                P(
                    seg.text,
                    cls=combine_classes(font_size.base, leading.relaxed, text_cls, m.b(3))
                ),
                
                # Footer with timing and source
                Div(
                    time_range,
                    duration_badge,
                    Div(cls=str(grow())),
                    source_info,
                    cls=combine_classes(
                        flex_display, items.center, gap(3),
                        font_size.xs
                    )
                ),
                cls=combine_classes(grow(), min_h(6))
            ),
            
            cls=combine_classes(
                card_body, p(4),
                flex_display, flex_direction.row, gap(3), items.start
            )
        ),
        
        # Absolutely positioned playing indicator
        playing_indicator,
        
        id=ReviewHtmlIds.review_card(seg.index),
        cls=combine_classes(
            card, "review-card",
            position.relative,
            bg_dui.base_100,
            w.full,
            transition.all, duration(150)
        ),
        data_segment_index=str(seg.index),
        data_start_time=str(chunk.start_time),
        data_end_time=str(chunk.end_time),
        data_card_role=card_role
    )

## Card Renderer Factory

Creates a callback compatible with the card stack library's `render_card` parameter.

In [ ]:
#| export
def create_review_card_renderer() -> Callable:  # Card renderer callback: (item, CardRenderContext) -> FT
    """Create a card renderer callback for review cards."""
    def _render(
        item:Any,  # AssembledSegment instance
        context:CardRenderContext,  # Render context from card stack library
    ) -> Any:  # Rendered review card component
        """Render a review card for the given item and viewport context."""
        return render_review_card(
            assembled=item,
            card_role=context.card_role,
        )
    return _render

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()